EXERCISE 1 - Use python datetime library to measure how long it took to 
send the data. To do so you need to import it

In [ ]:
import socket 

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
server_socket.bind(('localhost', 65432)) 
server_socket.listen(1) 
print("TCP Server is listening...") 


while True: 
    client_socket, client_address = server_socket.accept() 
    print(f"Connected to {client_address}") 
    data = client_socket.recv(1024) 
    print(f"Received: {data.decode()}") 
    client_socket.sendall(b"ACK: " + data) 
    client_socket.close() 

TCP Server is listening...
Connected to ('127.0.0.1', 58877)
Received: file_to_send.txt


EXERCISE 2: Change your script so that the data will be send using UDP 
and compare the time needed to send it using both protocols. 

In [ ]:
import socket


server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))

print("UDP Server is listening...")

while True:
    data, client_address = server_socket.recvfrom(1024)
    print(f"received from {client_address}: {data.decode()}")

    server_socket.sendto(b"ACK: " + data, client_address)


UDP Server is listening...
received from ('127.0.0.1', 49717): hellooooooooo


UDP Server is listening...<br>
received from ('127.0.0.1', 49717): hellooooooooo<br>

EXERCISE 3- SENDING TEXT FILE AND COMPARING TIME SEND BASED OFF OF SIZE OF FILE 

In [ ]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1)

print("📡 TCP Server is listening for file transfers...")

while True:
    client_socket, client_address = server_socket.accept()
    print(f"Connected to {client_address}")

    with open('received_file.txt', 'wb') as f:
        while True:
            data = client_socket.recv(1024)
            if not data:  
                break
            f.write(data)

    print("file received  and saved as 'received_file.txt")

    client_socket.sendall(b"file received")

    client_socket.close()



EXERCISE 4 - TXT FILE TRANSFER USING UDP

In [1]:
import socket

SERVER_HOST = 'localhost'
UDP_PORT = 65432  

server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind((SERVER_HOST, UDP_PORT))  

print("UDP Server is listening for file transfers...")

filename, client_address = server_socket.recvfrom(1024)
filename = filename.decode().strip()
print(f"receiving file: {filename} from {client_address}")


with open('received_file.txt', 'wb') as f:
    while True:
        data, _ = server_socket.recvfrom(1024)
        if data == b"EOF":  
            break
        f.write(data)

print("file received successfully and saved as 'received_file.txt'!")

server_socket.sendto(b"File received successfully.", client_address)


UDP Server is listening for file transfers...
receiving file: file_to_send.txt from ('127.0.0.1', 50357)
file received successfully and saved as 'received_file.txt'!


27

EXCERISE 1,2,3,4

In [ ]:
import socket
import threading
import datetime

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(2)  

print("TCP Server is listening...")

def handle_client(client_socket, client_address):
    print(f"[{datetime.datetime.now()}] Connected to {client_address}")

    try:
        client_socket.sendall(b"Hello Client!")

        while True:
            mode = client_socket.recv(1024).decode().strip()
            if not mode:
                break

            if mode == "CHAT":
                message = client_socket.recv(1024).decode().strip()
                if not message:
                    break
                print(f"[{datetime.datetime.now()}]  from {client_address}: {message}")

                response = f"ACK: {message}"
                client_socket.sendall(response.encode())

            elif mode == "FILE":
                filename = client_socket.recv(1024).decode().strip()
                if not filename:
                    break
                print(f"[{datetime.datetime.now()}]  file: {filename} from {client_address}")

                # Save file with a unique name
                save_path = f"received_from_{client_address[1]}_{filename}"
                with open(save_path, 'wb') as f:
                    while True:
                        file_data = client_socket.recv(1024)
                        if not file_data:
                            break
                        f.write(file_data)

                print(f"[{datetime.datetime.now()}] {filename} received from {client_address} and saved as {save_path}")
                client_socket.sendall(f"{filename} received ".encode())

    except ConnectionResetError:
        print(f"[{datetime.datetime.now()}] disconnected with: {client_address}")

    client_socket.close()

while True:
    client_socket, client_address = server_socket.accept()
    client_thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
    client_thread.start()


    

EXERCISE 5 - SERVER SIDE

In [ ]:
import socket
import threading

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(4)  

print("waiting for my clients to connect...")

clients = []
lock = threading.Lock() 


def handle_client(client_socket, client_id):

    while True:
        try:
            message = client_socket.recv(1024).decode()
            if not message or message.lower() == "exit":
                print(f"Client {client_id} disconnected")
                break

            print(f"Client {client_id} says: {message}")

            with lock:
                for i, client in enumerate(clients):
                    if i !=client_id:
                        try:
                            client.sendall(f"Client {client_id}: {message}".encode())
                        except:
                            print(f"Failed to send message to Client {i}")


        except (ConnectionResetError, BrokenPipeError):
            print(f"Client {client_id} disconnected ")
            break

    client_socket.close()
    with lock:
        clients[client_id] = None 

for i in range(4):
    client_socket, client_address = server_socket.accept()
    with lock:
        clients.append(client_socket)
    print(f"Client {i} connected from {client_address}")

    thread = threading.Thread(target=handle_client, args=(client_socket, i))
    thread.start()

server_socket.close()

waiting for my clients to connect...
Client 0 connected from ('127.0.0.1', 59367)
Client 1 connected from ('127.0.0.1', 59368)
Client 0 says: heloo
Client 1 says: heyyoo
Client 2 connected from ('127.0.0.1', 59458)
Client 2 says: hey im the 3rd client
Client 2 says: heyy
Client 1 says: heyo
Client 0 disconnected 


EXERCISE 6 - TCP SERVER FOR TWO CLIENTS THAT WANT TO TALK - EXERCISE 6, the server code ensures that client 0 recieves only client 1 message, and vice versa, more personal between the two.

In [ ]:

import socket
import threading


server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(2)  

print("Waiting for 2 clients to connect...")

clients = []

def handle_client(client_socket, client_id):
    other_client_id = 1 if client_id == 0 else 0  #

    while True:
        try:
            message = client_socket.recv(1024).decode()
            if not message or message.lower() == "exit":
                print(f"Client {client_id} disconnected")
                break

            print(f"Client {client_id} says: {message}")
            clients[other_client_id].sendall(f"Client {client_id}: {message}".encode())

        except (ConnectionResetError, BrokenPipeError):
            print(f"Client {client_id} disconnected ")
            break

    client_socket.close()

while len(clients) < 2:
    client_socket, client_address = server_socket.accept()
    clients.append(client_socket)
    print(f"Client {len(clients)-1} connected from {client_address}")

thread1 = threading.Thread(target=handle_client, args=(clients[0], 0))
thread2 = threading.Thread(target=handle_client, args=(clients[1], 1))
thread1.start()
thread2.start()

thread1.join()
thread2.join()

server_socket.close()


EXERCISE 7 AND 8 

In [ ]:
import socket
from cryptography.fernet import Fernet

key = Fernet.generate_key()
cipher = Fernet(key)

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1)

print("TCP Server is listening...")

while True:
    client_socket, client_address = server_socket.accept()
    print(f"Connected to {client_address}")

    client_socket.sendall(len(key).to_bytes(4, 'big'))
    client_socket.sendall(key) 

    encrypted_data = client_socket.recv(1024)
    if not encrypted_data:
        print("nothing")
        continue

    decrypted_message = cipher.decrypt(encrypted_data).decode()
    print(f"decrypted: {decrypted_message}")

    encrypted_response = cipher.encrypt(f"ACK: {decrypted_message}".encode())
    client_socket.sendall(encrypted_response)

    client_socket.close()
